In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf

In [2]:
train_data = pd.read_csv("/content/drive/MyDrive/Dataset/train_data.csv")
test_data = pd.read_csv("/content/drive/MyDrive/Dataset/test_data.csv")
validation_data = pd.read_csv("/content/drive/MyDrive/Dataset/validation_data.csv")

In [3]:
tk = tf.keras.preprocessing.text.Tokenizer(num_words=5000,oov_token="<oov>")

In [4]:
text=list(train_data["combined_text"])

In [5]:
tk.fit_on_texts(text)

In [6]:
import pickle
with open("/content/drive/MyDrive/Dataset/tokenizer_cnn_lstm.pickle","wb") as fp:
  pickle.dump(tk,fp,protocol=pickle.HIGHEST_PROTOCOL)

In [7]:
X_train = train_data["combined_text"]
X_test = test_data["combined_text"]
X_val = validation_data["combined_text"]

y_train = train_data["target"]
y_test = test_data["target"]
y_val = validation_data["target"]

In [8]:
X_train_seq = tk.texts_to_sequences(X_train)
X_test_seq = tk.texts_to_sequences(X_test)
X_val_seq = tk.texts_to_sequences(X_val)

X_train_seq_trunc = tf.keras.preprocessing.sequence.pad_sequences(X_train_seq, maxlen=100)
X_test_seq_trunc = tf.keras.preprocessing.sequence.pad_sequences(X_test_seq, maxlen=100)
X_val_seq_trunc = tf.keras.preprocessing.sequence.pad_sequences(X_val_seq, maxlen=100)

In [9]:
from tensorflow.keras import layers,models

In [10]:
emb_model = models.Sequential()
emb_model.add(layers.Embedding(len(tk.index_word), 8, input_length=100))
emb_model.add(layers.Convolution1D(16,4,activation='relu'))
emb_model.add(layers.AveragePooling1D())
emb_model.add(layers.Convolution1D(32,4,activation='relu'))
emb_model.add(layers.AveragePooling1D())
emb_model.add(layers.Flatten())
emb_model.add(layers.Dense(1, activation='sigmoid'))

In [11]:
emb_model.compile(optimizer='adam',loss=tf.keras.losses.BinaryCrossentropy() ,metrics=[tf.keras.metrics.BinaryAccuracy()])
emb_model.fit(x=X_train_seq_trunc,y=y_train,batch_size=128,epochs=10,validation_data=(X_test_seq_trunc,y_test))

Epoch 1/10
317/317 [==============================] - 33s 9ms/step - loss: 0.2817 - binary_accuracy: 0.8540 - val_loss: 0.1354 - val_binary_accuracy: 0.9532
Epoch 2/10
317/317 [==============================] - 2s 8ms/step - loss: 0.0998 - binary_accuracy: 0.9638 - val_loss: 0.1268 - val_binary_accuracy: 0.9568
Epoch 3/10
317/317 [==============================] - 2s 8ms/step - loss: 0.0737 - binary_accuracy: 0.9741 - val_loss: 0.1338 - val_binary_accuracy: 0.9572
Epoch 4/10
317/317 [==============================] - 2s 7ms/step - loss: 0.0593 - binary_accuracy: 0.9799 - val_loss: 0.1366 - val_binary_accuracy: 0.9537
Epoch 5/10
317/317 [==============================] - 2s 7ms/step - loss: 0.0490 - binary_accuracy: 0.9836 - val_loss: 0.1686 - val_binary_accuracy: 0.9532
Epoch 6/10
317/317 [==============================] - 2s 7ms/step - loss: 0.0404 - binary_accuracy: 0.9860 - val_loss: 0.1634 - val_binary_accuracy: 0.9586
Epoch 7/10
317/317 [==============================] - 2s 7ms/st

In [12]:
emb_model.save("/content/drive/MyDrive/Dataset/cnn_model.hdf5")

In [13]:
emb_model = models.Sequential()
emb_model.add(layers.Embedding(len(tk.index_word), 8, input_length=100))
emb_model.add(layers.LSTM(128,return_sequences=True))
emb_model.add(layers.LSTM(128,return_sequences=True))
emb_model.add(layers.Dense(1, activation='sigmoid'))

In [14]:
emb_model.compile(optimizer='adam',loss=tf.keras.losses.BinaryCrossentropy() ,metrics=[tf.keras.metrics.BinaryAccuracy()])
emb_model.fit(x=X_train_seq_trunc,y=y_train,batch_size=128,epochs=10,validation_data=(X_test_seq_trunc,y_test))

Epoch 1/10
317/317 [==============================] - 21s 54ms/step - loss: 0.3350 - binary_accuracy: 0.8431 - val_loss: 0.2769 - val_binary_accuracy: 0.8747
Epoch 2/10
317/317 [==============================] - 16s 51ms/step - loss: 0.2462 - binary_accuracy: 0.8913 - val_loss: 0.2649 - val_binary_accuracy: 0.8818
Epoch 3/10
317/317 [==============================] - 16s 51ms/step - loss: 0.2304 - binary_accuracy: 0.8979 - val_loss: 0.2571 - val_binary_accuracy: 0.8830
Epoch 4/10
317/317 [==============================] - 16s 51ms/step - loss: 0.2248 - binary_accuracy: 0.9006 - val_loss: 0.2623 - val_binary_accuracy: 0.8835
Epoch 5/10
317/317 [==============================] - 16s 51ms/step - loss: 0.2193 - binary_accuracy: 0.9032 - val_loss: 0.2602 - val_binary_accuracy: 0.8839
Epoch 6/10
317/317 [==============================] - 16s 51ms/step - loss: 0.2133 - binary_accuracy: 0.9062 - val_loss: 0.2632 - val_binary_accuracy: 0.8831
Epoch 7/10
317/317 [==============================] 

In [15]:
emb_model.save("/content/drive/MyDrive/Dataset/lstm_model.hdf5")